In [55]:
import os
import netCDF4
import logging
logging.getLogger().setLevel(logging.INFO)

In [27]:
def create_dataset(year, data_path):
    year = str(year)
    
    hgt_path = ''
    uwind_path = ''
    vwind_path = ''
    
    hgts_path = os.path.join(data_path,'hgts')
    uwinds_path = os.path.join(data_path,'u_winds')
    vwinds_path = os.path.join(data_path,'v_winds')
    
    for _,_, filenames in os.walk(hgts_path):
        if f"hgt.{year}.nc" in filenames:
            hgt_path = os.path.join(hgts_path,f"hgt.{year}.nc")
            
    for _,_, filenames in os.walk(uwinds_path):
        if f"uwnd.{year}.nc" in filenames:
            uwind_path = os.path.join(uwinds_path,f"uwnd.{year}.nc")
            
    for _,_, filenames in os.walk(vwinds_path):
        if f"vwnd.{year}.nc" in filenames:
            vwind_path = os.path.join(vwinds_path,f"vwnd.{year}.nc")
            
    if hgt_path and uwind_path and vwind_path:
        try:
            cdf = CDFData(hgt_path,uwind_path, vwind_path)
            return cdf
        except:
            logging.info(f'{year} IMPORT FAILURE')
    else:
        logging.info(f'Not all data found for {year}')

In [81]:
# 1979 - 2019

datasets = {}
for year in range(1979,2021):
    datasets[year] = create_dataset(year,DATA_PATH)

INFO:root: 17:46:14 - Retrieved 1979 hgt data
INFO:root: 17:46:18 - Retrieved 1979 uwnd data
INFO:root: 17:46:23 - Retrieved 1979 vwnd data
INFO:root: 17:46:32 - Retrieved 1980 hgt data
INFO:root: 17:46:37 - Retrieved 1980 uwnd data
INFO:root: 17:46:42 - Retrieved 1980 vwnd data
INFO:root: 17:46:50 - Retrieved 1981 hgt data
INFO:root: 17:46:55 - Retrieved 1981 uwnd data
INFO:root: 17:47:00 - Retrieved 1981 vwnd data
INFO:root: 17:47:08 - Retrieved 1982 hgt data
INFO:root: 17:47:13 - Retrieved 1982 uwnd data
INFO:root: 17:47:18 - Retrieved 1982 vwnd data
INFO:root: 17:47:26 - Retrieved 1983 hgt data
INFO:root: 17:47:31 - Retrieved 1983 uwnd data
INFO:root: 17:47:35 - Retrieved 1983 vwnd data
INFO:root: 17:47:44 - Retrieved 1984 hgt data
INFO:root: 17:47:49 - Retrieved 1984 uwnd data
INFO:root: 17:47:54 - Retrieved 1984 vwnd data
INFO:root: 17:48:02 - Retrieved 1985 hgt data
INFO:root: 17:48:07 - Retrieved 1985 uwnd data
INFO:root: 17:48:11 - Retrieved 1985 vwnd data
INFO:root: 17:48:20 

# Data retrieval technique
Want to be able to easily get back data for any year
```
hgts = data.get_hgts(year,mon,day)
```  
Data size for each year is not same -> Can't use numpy.ndarray
- Dictionary ?

In [80]:
def get_hgts(year, day=None, level=None):
    
    if isinstance(day, int):
        day -= 1  
    elif isinstance(day, tuple):
        day = date_to_day((year,*day))-1
    
    if level:
        assert day, 'Day must be provided along with level'
        level -=1
    
    return np.squeeze(datasets[year].hgts[day,level])

In [85]:
cdf = CDFData(r"E:\WeatherData\hgts\hgt.2014.nc",r"E:\WeatherData\u_winds\uwnd.2014.nc",r"E:\WeatherData\v_winds\vwnd.2014.nc")

INFO:root: 18:01:43 - Retrieved 2014 hgt data
INFO:root: 18:01:51 - Retrieved 2014 uwnd data


MemoryError: Unable to allocate 176. MiB for an array with shape (365, 12, 73, 144) and data type float32

In [2]:
from skimage.io import imread
from skimage.transform import resize
from skimage.filters import gaussian

In [40]:
DATA_ROOT = r"E:\WeatherData"

ref_date = (2012,10,28)

ref_u_winds_path = r"E:\WeatherData\u_winds\uwnd.2012.nc"
ref_v_winds_path = r"E:\WeatherData\v_winds\vwnd.2012.nc"
mask_path = r"C:/Users/Administrator/Downloads/continental_US_mask.png"

level = 6

lon0 = 220
lon1 = 340
lat0 = 20
lat1 = 70

ref_u = retrieve_loc_data(ref_u_winds_path,lon0,lon1,lat0,lat1)[date_to_day(ref_date)-1, level-1]
ref_v = retrieve_loc_data(ref_v_winds_path,lon0,lon1,lat0,lat1)[date_to_day(ref_date)-1, level-1]

mask_img = resize(imread(mask_path, as_gray=True),ref_u.shape[-2:])

In [41]:
def retrieve_loc_data(path, lon0, lon1, lat0, lat1):
    
    fh = Dataset(path, mode='r')
    lons = fh.variables['lon'][:].data
    lats = fh.variables['lat'][:].data
    
    if 'uwnd' in path:
        name = 'uwnd'
    else:
        name = 'vwnd'
    
    data = fh.variables[name][:].data
    
    lat_mask = ((lats >= lat0) & (lats <= lat1))

    lon_mask = ((lons >= lon0) & (lons <= lon1))

    start_lat = np.argwhere(lat_mask).min()
    end_lat = np.argwhere(lat_mask).max()

    start_lon = np.argwhere(lon_mask).min()
    end_lon = np.argwhere(lon_mask).max()

    loc_data = data[..., start_lat:end_lat+1, start_lon:end_lon+1]
    
    return loc_data


def get_score(u, v, ref_u, ref_v, mask):
    u *= mask
    u = normalize(u)
    v *= mask
    v = normalize(v)
    ref_u *= mask
    ref_u = normalize(ref_u)
    ref_v *= mask
    ref_v = normalize(ref_v)
    
    diff = np.sqrt(np.square(ref_u-u) + np.square(ref_v-v))
    
    return diff.sum()

In [49]:
#scores = {year1:[day1_score, day2_score, day3_score, ...]}
scores = {}

start_year = 2018
end_year = 2021

# for year in years:
for year in range(start_year, end_year):

    uwnd_path = os.path.join(DATA_ROOT,"u_winds",f'uwnd.{year}.nc')
    vwnd_path = os.path.join(DATA_ROOT,"v_winds",f'vwnd.{year}.nc')
    # retrieve data
    try:
        uwnds = retrieve_loc_data(uwnd_path,lon0,lon1,lat0,lat1)
        vwnds = retrieve_loc_data(vwnd_path,lon0,lon1,lat0,lat1)
    except:
        logging.info(f'Insufficient Data for {year}')
        print('Data Collection Fail')
        continue
    
    assert uwnds.shape == vwnds.shape
    
    scores[year] = []
    # for day in range(days):

    for day in range(uwnds.shape[0]):
        
        cur_u_winds = uwnds[day-1, level-1]
        cur_v_winds = vwnds[day-1, level-1]
        
        # scores[year].append(get_score(day))
        score = get_score(cur_u_winds, cur_v_winds, ref_u, ref_v, mask_img)
        scores[year].append(score)
    
    logging.info(f'Year {year} Complete')
# Delete scores[year] items with no entries/scores

In [51]:
logging.info('INFO')